## QSS-Compiler MLIR Python Bindings Demo

This notebook demos the qss-compiler MLIR Python Bindings. MLIR considers the python bindings to be an experimental feature. The're usage is very brittle at this point in time. Installation has not been automated. See notes at end of notebook for installation procedure. 

The current implementation does not support both the bindings and the qss-compiler's compile function due to an LLVM linkage issue. qss_compiler.compile has been disabled in this build.

In [1]:
import os
venv = os.path.basename(os.environ["VIRTUAL_ENV"])
if ( venv != 'qss-bindings-py3.10'):
    raise Exception(f"This notebook is running in the wrong virtual enviornment {venv}")

In [15]:
import numpy as np
from pathlib import Path

#### Import the MLIR bindings

These include the dialects included with MLIR as well as the additional dialects added by the qss-compiler

In [2]:
from mlir.ir import Context, InsertionPoint, Location, Module
from mlir.ir import F64Type, IntegerType, IndexType
from mlir.dialects import arith, builtin, std, scf

In [3]:
from qss_compiler.mlir.dialects import pulse, quir, complex # noqa: F401, E402
from qss_compiler.mlir.dialects import qcs,oq3

#### Create a demo waveform

Use numpy to create a short I/Q waveform

In [5]:
num_samples = 3
samples = np.arange(0,num_samples) + np.arange(0+1,num_samples+1) * 1j
samples_2d = np.zeros((samples.shape[0],2))
samples_2d[:,0] = np.real(samples)
samples_2d[:,1] = np.imag(samples)

#### Build the MLIR Module

In [6]:
with Context() as ctx:
    
    # register the custom qss-compiler dialects
    pulse.pulse.register_dialect()
    quir.quir.register_dialect()
    qcs.qcs.register_dialect()
    oq3.oq3.register_dialect()

    # get various types to be used later
    f64 = F64Type.get(ctx)
    i32 = IntegerType.get_signless(32)
    i1 = IntegerType.get_signless(1)

    idx = IndexType.get()
    
    frameType = pulse.FrameType.get(ctx)
    wf = pulse.WaveformType.get(ctx)
    
    # start creating the module - need to use a location
    with Location.unknown(ctx) as loc:
        
        # create the module
        module = Module.create(loc)

        # create a main function with an empty entry block
        with InsertionPoint(module.body):
            func = builtin.FuncOp("main", ([], [i32]))
            func.add_entry_block()
        
        # create a pulse.sequence with an empty entry block
        with InsertionPoint(module.body):
            seq1 = pulse.SequenceOp("seq_1",[wf, wf, frameType, frameType],[i1])
            seq1.add_entry_block()
            
            
        # fill the pulse.sequence
        with InsertionPoint(seq1.entry_block):
            gs, dg, mf1, mf2 = seq1.arguments
            c0 = arith.ConstantOp(i32, 656)
            pulse.DelayOp(mf1, c0)
            pulse.PlayOp(mf1, dg)
            zero = arith.ConstantOp(i1, 0)
            ret = pulse.ReturnOp(zero)

            
        # fill the main function 
        with InsertionPoint(func.entry_block):
            # start System
            qcs.SystemInitOp()

            # define constants
            c0 = arith.ConstantOp(f64, 0.0)
            c1_r = arith.ConstantOp(f64, 0.0)
            c1_i = arith.ConstantOp(f64, 0.0)
            c1_c = complex.CreateOp(c1_r, c1_i)
            
            # define Pulse Frames
            fM0 = pulse.Frame_CreateOp("drive")
            fMTRIGOUT = pulse.Frame_CreateOp("fMTRIGOUT")
            
            # create the for loop used for the shot-loop
            c2 = arith.ConstantOp(idx, 0)
            c3 = arith.ConstantOp(idx, 1000)
            c4 = arith.ConstantOp(idx, 1)
            
            loop = quir.ShotLoop(c2,c3,c4)
            
            # fill the shot loop
            with InsertionPoint(loop.body):
                
                # shot loop delay and init
                dur = quir.ConstantOp("duration","150us")
                quir.DelayOp(dur)
                si = qcs.ShotInitOp(1000)
                
                # create two waveforms
                gaussian_square_ice0 = pulse.Waveform_CreateOp(samples_2d)
                drag0 = pulse.Waveform_CreateOp(2* samples_2d)
                
                # call the sequence passing waveforms and frames
                res0 = pulse.CallSequenceOp([i1], "seq_1", [drag0, gaussian_square_ice0, fM0, fMTRIGOUT ])
                
                # scf.for requires a yield at the end
                scf.YieldOp(loop.inner_iter_args)
            
            # finalize and return from main func
            qcs.SystemFinalizeOp()
            c0_i = arith.ConstantOp(i32, 0)
            std.ReturnOp(c0_i)
print(module)

module {
  func @main() -> i32 {
    qcs.init
    %cst = arith.constant 0.000000e+00 : f64
    %cst_0 = arith.constant 0.000000e+00 : f64
    %cst_1 = arith.constant 0.000000e+00 : f64
    %0 = complex.create %cst_0, %cst_1 : complex<f64>
    %1 = "pulse.create_frame"() {uid = "drive"} : () -> !pulse.frame
    %2 = "pulse.create_frame"() {uid = "fMTRIGOUT"} : () -> !pulse.frame
    %c0 = arith.constant 0 : index
    %c1000 = arith.constant 1000 : index
    %c1 = arith.constant 1 : index
    scf.for %arg0 = %c0 to %c1000 step %c1 {
      %dur = quir.constant #quir.duration<"150us" : !quir.duration>
      quir.delay %dur, () : !quir.duration, () -> ()
      qcs.shot_init {qcs.numShots = 1000 : i32, quir.startNCOs = false}
      %3 = pulse.create_waveform dense<[[0.000000e+00, 1.000000e+00], [1.000000e+00, 2.000000e+00], [2.000000e+00, 3.000000e+00]]> : tensor<3x2xf64> -> !pulse.waveform
      %4 = pulse.create_waveform dense<[[0.000000e+00, 2.000000e+00], [2.000000e+00, 4.000000e+00], [4

#### Demo using MLIR Python Bindings for sqore translation

In [7]:
import qss_pulsegen.sqore2awg as sqore2awg
from qss_exp_params.exp_params import Exp_Params

In [9]:
exp_params = Path('~/awg/exp_params.yaml').expanduser()
sq2_file = str(Path('~/scratch/pulse_layer/mvp1/single_qubit_simple.sq2').expanduser())

In [ ]:
exp_params = Exp_Params(exp_params, add_qmap=True)
exp_params.load()
sqore2awg.synchronous_process_sq(sq2_file,
                                exp_params.name,
                                plot=False,
                                generate=True,
                                exp_params=exp_params,
                                random_dir=False,
                                cleanup=False,
                                codegen="qss-compiler",
                                )

qss-compiler is unavailable on Apple Arm, writing mlir to test_out.mlir.
translate_payload was not passed a qem_filename returning


module {
  func @main() -> i32 {
    qcs.init
    %cst = arith.constant 0.000000e+00 : f64
    %0 = complex.create %cst, %cst : complex<f64>
    %angle = quir.constant #quir.angle<0.000000e+00 : !quir.angle<20>>
    %1 = "pulse.create_frame"() {uid = "M1"} : () -> !pulse.frame
    %2 = "pulse.create_frame"() {uid = "MTRIGOUT.M1"} : () -> !pulse.frame
    %3 = "pulse.create_frame"() {uid = "Q0"} : () -> !pulse.frame
    %4 = "pulse.create_frame"() {uid = "Q1"} : () -> !pulse.frame
    %5 = "pulse.create_frame"() {uid = "Q2"} : () -> !pulse.frame
    %6 = "pulse.create_frame"() {uid = "Q3"} : () -> !pulse.frame
    %7 = "pulse.create_frame"() {uid = "Q4"} : () -> !pulse.frame
    %8 = "pulse.create_frame"() {uid = "Q5"} : () -> !pulse.frame
    %9 = "pulse.create_frame"() {uid = "Q6"} : () -> !pulse.frame
    %c0 = arith.constant 0 : index
    %c1000 = arith.constant 1000 : index
    %c1 = arith.constant 1 : index
    scf.for %arg0 = %c0 to %c1000 step %c1 {
      %dur = quir.constant #q

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [12]:
with Context() as ctx:
    with Location.unknown(ctx) as loc:
        idx = IndexType.get()
        c2 = arith.ConstantOp(idx, 0)
        c3 = arith.ConstantOp(idx, 1000)
        c4 = arith.ConstantOp(idx, 1)
        forLoop = scf.ForOp(c2, c3, c4)
        print(dir(forLoop))

['OPERATION_NAME', '_CAPIPtr', '_ODS_OPERAND_SEGMENTS', '_ODS_REGIONS', '_ODS_RESULT_SEGMENTS', '_Raw', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__pybind11_module_local_v4_clang_libcpp_cxxabi1002__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'attributes', 'body', 'build_generic', 'context', 'detach_from_parent', 'get_asm', 'induction_variable', 'initArgs', 'inner_iter_args', 'location', 'lowerBound', 'move_after', 'move_before', 'operands', 'operation', 'print', 'region', 'regions', 'result', 'results', 'results_', 'step', 'upperBound', 'verify']


In [13]:
dir(oq3)

['AngleAddOp',
 'AngleCmpOp',
 'AngleDivOp',
 'AngleMulOp',
 'AngleSubOp',
 'AssignArrayElementOp',
 'CBitAndOp',
 'CBitAssignBitOp',
 'CBitExtractBitOp',
 'CBitInsertBitOp',
 'CBitLShiftOp',
 'CBitNotOp',
 'CBitOrOp',
 'CBitPopcountOp',
 'CBitRShiftOp',
 'CBitRotLOp',
 'CBitRotROp',
 'CBitXorOp',
 'CastOp',
 'DeclareArrayOp',
 'DeclareStretchOp',
 'DeclareVariableOp',
 'UnitAttr',
 'UseArrayElementOp',
 'VariableAssignOp',
 'VariableLoadOp',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'builtins',
 'oq3',
 'scf']

Using this notebook requires manual install of MLIR python bindings:
1. Copy or link to `qss-compiler/third_party/mlir/pyproject.toml` from `~/.conan/data/llvm/qss/stable/<hash>/python_packages/mlir_core`
2. Run `pip install -e .`
3. Copy or link to `~/.conan/data/llvm/qss/stable/<hash>/python_packages/mlir_core/mlir/_mlir_libs/libMLIRPythonCAPI.dylib` from `<python>/lib`
4. build compiler
5. cd `<build> qss_compiler/python_lib`
6. alias gsed if required
6. run `bash setup_mlir.sh`
7. run `pip install -e .`

Need to link to correct libMLIRPythonCAPI.dylib from /usr/local/lib
```
ls -s ~/.conan/data/<llvm-version/qss/stable/package/<hash>/python_packages/mlir_core/mlir/_mlir_libs/libMLIRPythonCAPI.dylib
```

This currently only works with a release build of LLVM due to asserts being triggered in the debug build